In [30]:
import cv2
import numpy as np

img = cv2.imread('image/street.jpg')
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
    # 프레임 읽기 (ret : 성공 여부, frame: 이미지)
    # ret, frame = cap.read()
    # 속도 향상을 위해 영상크기를 절반으로 축소
    # frame = cv2.resize(frame, None, fx=0.5, fy=0.5, \
                        # interpolation=cv2.INTER_AREA)
    # 캠화면을 끄기 위한 방법
    # if cv2.waitKey(1) == 27: # esc 키로 종료
        # break

# 그레이 스케일로 변환 : 단순하게 만들기 위함
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 노이즈를 제거하는 작업(가우시안 필터 : 라플라시안 필터를 사용하기 전 항상 실행)
img_gray = cv2.GaussianBlur(img_gray, (9,9), 0)
# 라플라시안 필터 적용
edges = cv2.Laplacian(img_gray, -1, None, 5)
# 전역 스레시홀딩을 사용해서 경계값을 뚜렷하게 만들기
ret, sketch = cv2.threshold(edges, 80, 255, cv2.THRESH_BINARY_INV)

# 경계값을 강조하는 팽창 연산
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
sketch_2 = cv2.erode(sketch, kernel)
# 경계값을 자연스럽게 하는 미디언 블러 사용
sketch = cv2.medianBlur(sketch_2, 5)
# 이미지의 사이즈를 3차원으로 변경
img_sketch = cv2.cvtColor(sketch, cv2.COLOR_GRAY2BGR)
# 이미지에 컬러 적용
img_paint = cv2.blur(img, (7,7))
img_paint = cv2.bitwise_and(img_paint, img_paint, mask=sketch)

merged = np.hstack((img_sketch, img_paint))

    # cv2.imshow('Sketch Camera', frame)
cv2.imshow('Sketch Camera', merged)
cv2.waitKey(0)
# cap.release() # 외부장치 접근 해제
cv2.destroyAllWindows()

## 틀린그림찾기

In [50]:
import cv2
import numpy as np

# 이미지 확인해보기
img = cv2.imread('diff/diff_4.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
re_img = cv2.resize(img, None, fx=0.7, fy=0.7, interpolation=cv2.INTER_AREA)
# 이미지 나눠주는 작업
# 이미지 합쳐서 경계값이나 바이너리 이미지 찾기
# 다른 부분의 위치값을 표시하기


cv2.imshow('diff', re_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
h = int(re_img.shape[0]/2)

img_1 = re_img[:h, :]
img_2 = re_img[h:, :]

cv2.imshow('diff', img_1)
cv2.imshow('diff2', img_2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [54]:
img_1.shape

(504, 466)

In [68]:
# 이미지에서 검정색 배경을 제거하는 작업
ht = np.where((img_1 == 0).sum(axis=1) <= 50)[0][0]
hb = np.where((img_1 == 0).sum(axis=1) <= 50)[0][-1]

In [106]:
def img_rec(path):
    # 이미지 불러오기 & 이미지 크기 조정
    img = cv2.imread(path)    
    re_img = cv2.resize(img, None, fx=0.7, fy=0.7, interpolation=cv2.INTER_AREA)
    # 이미지 처리를 쉽게 하기 위해서 gray로 변환
    
    # 이미지 두 개로 분할
    h = int(re_img.shape[0]/2)
    img_1 = re_img[:h, :]
    img_2 = re_img[h:, :]

    # 이미지에서 검정색 배경을 제거하는 작업
    imgs=[img_1, img_2]
    crop_imgs=[]
    for i in range(2):
        img_gray = cv2.cvtColor(imgs[i], cv2.COLOR_BGR2GRAY)
        ht = np.where((img_gray == 0).sum(axis=1) <= 50)[0][0]
        hb = np.where((img_gray == 0).sum(axis=1) <= 50)[0][-1]
        crop_imgs.append(imgs[i][ht:hb])

    return tuple(crop_imgs)

img_source = ['diff_1.jpg', 'diff_2.jpg', 'diff_3.jpg', 'diff_4.jpg', 'diff_5.jpg']
for i in range(len(img_source)):
    img1, img2 = img_rec('diff/' + img_source[i])
    if img1.shape[0] != img2.shape[0]:
        img2 = img2[:-1,:,:]
    cv2.imwrite('diff/'+ f'upper_{i}.jpg', img1)
    cv2.imwrite('diff/'+ f'bottom_{i}.jpg', img2)

# img1, img2 = img_rec('diff/diff_5.jpg')

# cv2.imshow('diff_a', img1)
# cv2.imshow('diff_b', img2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [120]:
img1 = cv2.imread('diff/upper_4.jpg')
img2 = cv2.imread('diff/bottom_4.jpg')
print(img1.shape, img2.shape)

cv2.imshow('diff_a', img1)
cv2.imshow('diff_b', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

(375, 466, 3) (375, 466, 3)


In [152]:
# 이미지 차이 계산 식
img1 = cv2.imread('diff/upper_1.jpg')
img2 = cv2.imread('diff/bottom_1.jpg')

diff = cv2.absdiff(img1, img2)

In [154]:
# 2차원으로 변환
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
# 차이 강조를 위해서 쓰래쉬홀드를 진행
_, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)

# 컨투어 찾기
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

img1 = cv2.imread('diff/upper_1.jpg')
img2 = cv2.imread('diff/bottom_1.jpg')

for cnt in contours:
    # 컨투어를 감싸는 최소 사각형 계산
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 2 and h > 2:
        cv2.rectangle(img1, (x, y), (x+w, y+h), (255,255,255), 4)

diff = cv2.absdiff(img1, img2)
gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

img1 = cv2.imread('diff/upper_1.jpg')
img2 = cv2.imread('diff/bottom_1.jpg')

for cnt in contours:
    # 컨투어를 감싸는 최소 사각형 계산
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 2 and h > 2:
        cv2.rectangle(img1, (x, y), (x+w, y+h), (255,255,0), 3)

# print(contours)
cv2.imshow('diff', img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [213]:
def diff_img(k):
    # 번호에 맞는 이미지 가져오기
    img1 = cv2.imread(f'diff/upper_{int(k)}.jpg')
    img2 = cv2.imread(f'diff/bottom_{int(k)}.jpg')
    # 차이값 계산
    diff = cv2.absdiff(img1, img2)
    # thresh hold
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    # 차이 강조를 위해서 쓰래쉬홀드를 진행
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)    
    # 컨투어 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 3 and h > 3:
            cv2.rectangle(img1, (x, y), (x+w, y+h), (255,255,255), 10)

    # 두번째 diff 작업
    # img_after = img1.copy() 
    # img2 = cv2.imread(f'diff/bottom_{int(x)}.jpg')
    # 차이값 계산
    diff = cv2.absdiff(img1, img2)
    # thresh hold
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    # 차이 강조를 위해서 쓰래쉬홀드를 진행
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)    
    # 컨투어 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    real_img = cv2.imread(f'diff/upper_{int(k)}.jpg')
    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 5 and h > 5:
            cv2.rectangle(real_img, (x, y), (x+w, y+h), (170,0,200), 2)
            
    return real_img

In [216]:
img = diff_img(2)
cv2.imshow('diff', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [223]:
# 1. 이미지 분할 코드
# 보관된 이미지들
img_source = ['diff_1.jpg', 'diff_2.jpg', 'diff_3.jpg', 'diff_4.jpg', 'diff_5.jpg']
# 보관된 이미지를 반복해서 수정
for i in range(len(img_source)):
    # 이미지를 분할하는 img_rec 사용자 정의함수를 이용
    img1, img2 = img_rec('diff/' + img_source[i])
    # 이미지 크기(height)가 다르면, 마지막 한 줄을 삭제
    if img1.shape[0] != img2.shape[0]:
        img2 = img2[:-1,:,:]
    # 분할된 이미지를 저장하는 코드
    cv2.imwrite('diff/'+ f'upper_{i}.jpg', img1)
    cv2.imwrite('diff/'+ f'bottom_{i}.jpg', img2)

# 2. 틀린 부분 찾고 그림 그리기
# 2-1. 처음에는 img1과 img2의 diff를 만들고
# 2-2. 차이값에 대한 img1의 사각형을 그림
# 2-3. 사각형의 두께가 크면 클수록 사각형 겹침이 생김
# 2-4. 2-3에서 만든 img1에다가 다시 한번 diff작업을 진행
# 2-5. 더 큰 사각형을 만들어줌
def diff_img(k):
    # 번호에 맞는 이미지 가져오기
    img1 = cv2.imread(f'diff/upper_{int(k)}.jpg')
    img2 = cv2.imread(f'diff/bottom_{int(k)}.jpg')
    # 차이값 계산
    diff = cv2.absdiff(img1, img2)
    # thresh hold
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    # 차이 강조를 위해서 쓰래쉬홀드를 진행
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)    
    # 컨투어 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        # 사각형을 굵기를 두껍게 주는 이유는 겹쳐야 하나로 인식하기 쉬워서
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 3 and h > 3:
            cv2.rectangle(img1, (x, y), (x+w, y+h), (255,255,255), 10)
    # 사각형이 그려진 이미지를 가지고 더 큰 사각형을 그리는 작업
    # 두번째 diff 작업
    # 차이값 계산
    diff = cv2.absdiff(img1, img2)
    # thresh hold
    # 2차원으로 변환
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    # 차이 강조를 위해서 쓰래쉬홀드를 진행
    _, thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)    
    # 컨투어 찾기
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img1 = cv2.imread(f'diff/upper_{int(k)}.jpg')
    for cnt in contours:
        # 컨투어를 감싸는 최소 사각형 계산
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 5 and h > 5:
            cv2.rectangle(img1, (x, y), (x+w, y+h), (170,0,200), 2)
            
    return img1, img2

input_num = input("0~4까지의 숫자를 입력하시오.")
img_1, img_2 = diff_img(input_num)
original_u = cv2.imread(f'diff/upper_{int(input_num)}.jpg')
original_b = cv2.imread(f'diff/bottom_{int(input_num)}.jpg')
merged = np.hstack((original_u, original_b))
cv2.imshow("틀린그림찾기 원본이미지", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

merged = np.hstack((img_1, img_2))
cv2.imshow("틀린그림찾기 완료", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

0~4까지의 숫자를 입력하시오. 4
